In [1]:
import pandas as pd
import pickle

In [2]:
# Load the saved XGBoost model
with open('best_xgb_model.pkl', 'rb') as f:
    best_xgb = pickle.load(f)

# Load the saved scaler
with open('scaler.pkl', 'rb') as f:
    scaler = pickle.load(f)

# Load the saved label encoder
with open('label_encoder.pkl', 'rb') as f:
    label_encoder = pickle.load(f)

print("Models and encoders loaded successfully.")


FileNotFoundError: [Errno 2] No such file or directory: 'best_xgb_model.pkl'

In [8]:
from dotenv import load_dotenv
import os
from pathlib import Path

# Load environment variables
load_dotenv()

# Get dataset directory
dataset_dir = os.getenv("DATASET_DIR")
if dataset_dir is None:
    raise ValueError("DATASET_DIR not found in .env file")

# Use pathlib for reliable path handling
data_file = Path(dataset_dir) / "2_earthquake_1995-2023.csv"
df = pd.read_csv(data_file)

In [9]:
# Select the features from the new dataframe (same as training data)
# Select features
selected_features = [
    'dmin', 'cdi', 'sig', 'tsunami', 'gap'
]

# Preprocess the new data
X_new = df[selected_features]

# Scale the features using the saved scaler
X_new_scaled = scaler.transform(X_new)

# Predict missing 'alert' values
missing_alert_indices = df['alert'].isna()  # Identify rows where 'alert' is missing
X_missing = X_new_scaled[missing_alert_indices]  # Features for missing alert rows

# Use the XGBoost model to predict missing 'alert' values
predicted_alerts = best_xgb.predict(X_missing)

# Map the predictions back to the original labels using label encoder
predicted_alerts = label_encoder.inverse_transform(predicted_alerts)

# Fill the missing 'alert' values with the predictions
df.loc[missing_alert_indices, 'alert'] = predicted_alerts

print("Missing 'alert' values have been filled.")


Missing 'alert' values have been filled.


In [10]:
df.drop(["Unnamed: 0.1","Unnamed: 0"],inplace = True,axis = 1)
# Save the updated dataframe with filled 'alert' values
df.to_csv('3_earthquake_1995-2023.csv', index=False)

print("Updated dataframe saved'")


Updated dataframe saved'
